In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras import backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import time

# **Functions**

In [ ]:
def import_resnet50v2():
    # importing resnet50v2 from keras
    model = keras.applications.ResNet50V2(
        include_top=False,
        weights="imagenet",
        input_shape=(32,32,3),
        pooling='avg'
    )
    
    print("\nResnet50V2 has been imported")
    
    return model

In [ ]:
# resnet50v2_pretrained.summary()

In [ ]:
# 15TH BATCH TRAINING AND ONWARDS, LAYERS FROZE TILL: conv5_block2_preact_bn
# 12TH till 14th BATCH TRAINING, LAYERS FROZE TILL: conv5_block3_preact_bn
# 1ST TILL 11TH BATCH TRAINING, ALL LAYERS FROZE

# freezing resnet50v2 layers after conv5_block2

def freeze_layers(model, layer='none'):
    
    if layer == 'none':

        # freezing models all layers
        for layer in model.layers:
            layer.trainable = False
            
        # Assuming your model is named 'model'
        trainable_count = np.sum([np.prod(w.shape) for w in model.trainable_weights])
            
        print("All layers froze")
        print(f"Trainable parameters: {trainable_count}\n")
    
    else:

        # Find the index of the layer you want to stop freezing at
        stop_layer_name = layer
        stop_layer_index = None
        for i, layer in enumerate(model.layers):
            if stop_layer_name in layer.name:
                stop_layer_index = i
                break

        # Freeze layers up to the stop layer
        for layer in model.layers[:stop_layer_index + 1]:
            layer.trainable = False

        # Unfreeze layers after the stop layer
        for layer in model.layers[stop_layer_index + 1:]:
            layer.trainable = True

        # Assuming your model is named 'model'
        trainable_count = np.sum([np.prod(w.shape) for w in model.trainable_weights])

        print(f"All layers froze prior to '{stop_layer_name}'")
        print(f"Trainable parameters: {trainable_count}\n")
    
    return model

In [ ]:
# transofrming data for training

def transform_data_for_training(file_path):
    
    # importing training data
    train_data = pd.read_csv(file_path)

    # specifying size of the image
    img_size_org = (28,28,1)

    # separating labels
    train_targets = train_data['label']
    train_features = train_data.drop('label', axis=1)
    
    # converting data to int type
    train_targets = train_targets.astype(int)
    train_features = train_features.astype(int)

    # one-hot-encoding targets
    train_targets = pd.get_dummies(train_targets, columns=['label'])

    # normalizing features
    train_features = train_features / 255

    # converting pandas dataframe to numpy array
    train_targets = train_targets.values
    train_features = train_features.values

    # resizing orignal images dimension and number of channels to the required dimensions and number of channels
    train_features = train_features.reshape((train_features.shape[0],28,28))
    train_features = np.expand_dims(train_features, axis=-1)
    train_features = tf.image.grayscale_to_rgb(tf.convert_to_tensor(train_features))
    train_features = tf.image.resize(train_features, (32,32))
    train_features = train_features.numpy()
    
    return train_features, train_targets

In [ ]:
# transofrming data for error analysis

def transform_data_for_erroranalysis(file_path, training_ratio, random_seed):
    
    # importing training data
    train_data = pd.read_csv(file_path)

    # specifying size of the image
    img_size_org = (28,28,1)

    # separating labels
    train_targets = train_data['label']
    train_features = train_data.drop('label', axis=1)

    # one-hot-encoding targets
    train_targets = pd.get_dummies(train_targets, columns=['label'])

    # normalizing features
    train_features = train_features / 255

    # added original index to training features
    train_features['org_index'] = train_features.index

    # converting pandas dataframe to numpy array
    train_targets = train_targets.values
    train_features = train_features.values

    # splitting data into trainning and validation sets
    X_train, X_val, y_train, y_val = train_val_split(train_features, train_targets, training_ratio, random_seed)

    # Find the indices of True values in each row and concatenate into a flat list
    y_val_trueindices = np.concatenate([np.where(row)[0] for row in y_val])

    # Convert the NumPy array to a Pandas DataFrame
    X_val_pd = pd.DataFrame(X_val)
    y_val_pd = pd.DataFrame(y_val_trueindices)

    # Rename the last column to 'label'
    y_val_pd = y_val_pd.rename(columns={y_val_pd.columns[-1]: 'label'})

    # Concatenate them along the columns
    merged_df = pd.concat([X_val_pd, y_val_pd], axis=1)

    # Sort the DataFrame based on the 'label' column in ascending order
    sorted_merged_df = merged_df.sort_values(by='label')

    # separating necessary columns
    y_val_pd = sorted_merged_df['label']
    X_val_pd = sorted_merged_df.drop(['label', 784], axis=1)
    org_index = sorted_merged_df[784]
    X_train = np.delete(X_train, -1, 1)
    
    # one-hot-encoding labels
    y_val_pd = pd.get_dummies(y_val_pd, columns=['label'])

    # converting pandas dataframe to numpy array
    y_val = y_val_pd.values
    X_val = X_val_pd.values
    org_index = org_index.values

    # resizing orignal images dimension and number of channels to the required dimensions and number of channels
    X_val = X_val.reshape((4200,28,28))
    X_val = np.expand_dims(X_val, axis=-1)
    X_val = tf.image.grayscale_to_rgb(tf.convert_to_tensor(X_val))
    X_val = tf.image.resize(X_val, (32,32))
    X_val = X_val.numpy()

    X_train = X_train.reshape((37800,28,28))
    X_train = np.expand_dims(X_train, axis=-1)
    X_train = tf.image.grayscale_to_rgb(tf.convert_to_tensor(X_train))
    X_train = tf.image.resize(X_train, (32,32))
    X_train = X_train.numpy()

    return X_train, X_val, y_train, y_val, org_index

In [ ]:
# splitting training data

def train_val_split(train_features, train_targets, training_ratio, random_seed):
    
    # setting a fixed seed for reproducibility
    np.random.seed(random_seed)
    
    # splitting data between train, dev, and test sets
    training_ratio = 0.9
    X_train, X_val, y_train, y_val = train_test_split(train_features, train_targets, test_size=(1-training_ratio))
    
    return X_train, X_val, y_train, y_val

In [ ]:
# creating resnet50v2 with added layers

def resnet50v2_custom_create(resnet50v2_pretrained, model_name, random_seed, learningrate, momentum, dropoutrate, L2_regularizer):
        
    # building the model
    model = Sequential(name=model_name)
    
    # adding final layers to train
    model.add(resnet50v2_pretrained)
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal(seed=random_seed), use_bias=True, bias_initializer='zeros', kernel_regularizer=tf.keras.regularizers.L2(L2_regularizer)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropoutrate))
    model.add(Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal(seed=random_seed), use_bias=True, bias_initializer='zeros', kernel_regularizer=tf.keras.regularizers.L2(L2_regularizer)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropoutrate))
    model.add(Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal(seed=random_seed), use_bias=True, bias_initializer='zeros', kernel_regularizer=tf.keras.regularizers.L2(L2_regularizer)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropoutrate))
    model.add(Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal(seed=random_seed), use_bias=True, bias_initializer='zeros', kernel_regularizer=tf.keras.regularizers.L2(L2_regularizer)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropoutrate))
    model.add(Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal(seed=random_seed), use_bias=True, bias_initializer='zeros', kernel_regularizer=tf.keras.regularizers.L2(L2_regularizer)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropoutrate))
    model.add(Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal(seed=random_seed), use_bias=True, bias_initializer='zeros', kernel_regularizer=tf.keras.regularizers.L2(L2_regularizer)))
    model.add(Dense(10, activation='softmax'))

    # printing models summary
    model.summary()

    # setting up learning rate decay
    # lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(learningrate, decay_steps=X_train.shape[0]/batchsize, decay_rate=lrdecay, staircase=False)

    # compiling model and setting hyperparameters
    model.compile(optimizer=Adam(learning_rate=learningrate, beta_1=momentum), loss='categorical_crossentropy', metrics=['accuracy'])
    
    trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])
    total_params = trainable_count + non_trainable_count
    
    return model, total_params, trainable_count, non_trainable_count

In [ ]:
# training resnet50v2 with added layers

def resnet50v2_custom_train(model, X_train, y_train, X_val, y_val, batchsize, epoch, directory_path_output):
        
    checkpoint_path = f"{directory_path_output}{model.name}_21st_batch_training.h5"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

    # Record the start time
    start_time = time.time()

    # training model
    history = model.fit(X_train, y_train, epochs=epoch, batch_size=batchsize, validation_data=(X_val, y_val), callbacks=[cp_callback])

    # Record the end time
    end_time = time.time()

    # Calculate the elapsed time
    training_time = end_time - start_time

    # Format the time as hours, minutes, and seconds
    training_time_formatted = time.strftime("%H:%M:%S", time.gmtime(training_time))
        
    return history, training_time_formatted

In [ ]:
def plot_graph(history, model_name, training_time_formatted, directory_path_output):
    
    # Plotting/Saving Graphs
    # Extract the training history
    train_loss = history.history['loss']
    train_accuracy = history.history['accuracy']
    val_loss = history.history['val_loss']
    val_accuracy = history.history['val_accuracy']

    # Plot training and validation loss
    plt.figure(figsize=(16, 12))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss')
    plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Training Accuracy')
    plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    legend_text = f'Model Name: {model_name}\nLearning Rate: {learningrate:.5f}\nMomentum: {momentum:.3f}\nDropout Rate: {dropoutrate:.3f}\nL2 Regularizer: {L2_regularizer:.4f}\nBatch Size: {batchsize}\nEpochs: {epoch}\nTraining Time: {training_time_formatted}'
    plt.figtext(0.01, 0.01, legend_text, fontsize=10, va="bottom", ha="left")

    plt.tight_layout()

    # Save the figures to the specified directory
    figure_name = f"{model_name}_lowest_val_loss_{min(history.history['val_loss']):.4f}_21st_batch_training.png"  # Replace with your desired file name
    figure_path = os.path.join(directory_path_output, figure_name)
    plt.savefig(figure_path)
    plt.close()  # Close the figure to release resources

# **Setting Hyperparameter Values**

In [ ]:
# LAST RAN: 14th Batch

# specifying a fixed seed for reproducibility
random_seed = 13

# file path of training data and directory path of output data
file_path_input = "/kaggle/input/digit-recognizer/train.csv"
directory_path_output = "/kaggle/working/"

# training ratio to split data on
training_ratio = 0.9

# hyperparameter values
learningrates = [0.001, 0.001, 0.001, 0.002, 0.002, 0.002, 0.003, 0.003, 0.003]
momentums = [0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95]
dropoutrates = [0.3, 0.4, 0.5, 0.3, 0.4, 0.5, 0.3, 0.4, 0.5]
L2_regularizers = [0.01, 0.01, 0.01, 0.001, 0.001, 0.001, 0.0001, 0.0001, 0.0001]
batchsizes = [256, 256, 256, 256, 256, 256, 256, 256, 256]
epoch = 100

# transforming training data for training
train_features, train_targets = transform_data_for_training(file_path_input)

#transforming augmented data for training
X_train_aug_1, y_train_aug_1 = transform_data_for_training("/kaggle/input/digit-recognizer-augmented-datasets/augmented_data_resnet50v2_5_17th_batch_training_5029.csv")
X_train_aug_2, y_train_aug_2 = transform_data_for_training("/kaggle/input/digit-recognizer-augmented-datasets/augmented_data_resnet50v2_5_18th_batch_training.csv")
X_train_aug_3, y_train_aug_3 = transform_data_for_training("/kaggle/input/digit-recognizer-augmented-datasets/augmented_data_resnet50v2_9_19th_batch_training_2520.csv")

# splitting data into trainning and validation sets
X_train_non_aug, X_val, y_train_non_aug, y_val = train_val_split(train_features, train_targets, training_ratio, random_seed)

# adding augmented data to training data
X_train = np.concatenate((X_train_non_aug, X_train_aug_1, X_train_aug_2, X_train_aug_3), axis=0)
y_train = np.concatenate((y_train_non_aug, y_train_aug_1, y_train_aug_2, y_train_aug_3), axis=0)

# new changes that are made for each training batch
new_changes = "Added 2500 aditional augmented images to training data"

# printing hyperparameter values and data distribution
print("Random Seed: ", random_seed)
print("\nHyperparameter Values:")
print("Learning Rates: ", learningrates)
print("Momentums: ", momentums)
print("Dropout Rates: ", dropoutrates)
print("L2 Regularizers: ", L2_regularizers)
print("Batch Sizes: ", batchsizes)
print("Epochs: ", epoch)
print("\nData Distribution:")
print(f"Total Data: {train_features.shape[0]+X_train_aug_1.shape[0]+X_train_aug_2.shape[0]}")
print(f'training features = {X_train.shape}')
print(f'training targets = {y_train.shape}')
print(f'validation features = {X_val.shape}')
print(f'validation targets = {y_val.shape}')
# print(f'test features = {X_test.shape}')
# print(f'test targets = {y_test.shape}')
print(f'X_train = {(X_train.shape[0]/(train_features.shape[0]+X_train_aug_1.shape[0]+X_train_aug_2.shape[0]+X_train_aug_3.shape[0]))*100:.2f}%')
print(f'X_val = {(X_val.shape[0]/(train_features.shape[0]+X_train_aug_1.shape[0]+X_train_aug_2.shape[0]+X_train_aug_3.shape[0]))*100:.2f}%')
# print(f'X_test = {(X_test.shape[0]/train_features.shape[0])*100:.2f}%')
print(f'y_train = {(y_train.shape[0]/(train_targets.shape[0]+X_train_aug_1.shape[0]+X_train_aug_2.shape[0]+X_train_aug_3.shape[0]))*100:.2f}%')
print(f'y_val = {(y_val.shape[0]/(train_targets.shape[0]+X_train_aug_1.shape[0]+X_train_aug_2.shape[0]+X_train_aug_3.shape[0]))*100:.2f}%')
# print(f'y_test = {(y_test.shape[0]/train_targets.shape[0])*100:.2f}%')

# **Training Model**

In [ ]:
# run this code cell to train and save models stats

modelnames = []
totalparameters = []
trainableparameters = []
nontrainableparameters = []

for i in range (0, len(learningrates)):
    
    modelname = f"resnet50v2_{i+1}"
    modelnames.append(modelname)
    
    # hyperparameter values
    learningrate = learningrates[i]
    momentum = momentums[i]
    dropoutrate = dropoutrates[i]
    L2_regularizer = L2_regularizers[i]
    batchsize = batchsizes[i]
    
    # importing renset50v2 pretrained 
    resnet50v2_pretrained = import_resnet50v2()
        
    # freezing layers of resnet50v2
    first_trainable_layer = 'conv5_block2_preact_bn'
    resnet50v2_pretrained = freeze_layers(resnet50v2_pretrained, 'conv5_block2_preact_bn')
    
    # create resnet50v2 with added layers
    model, totalparams, trainableparams, nontrainableparams = resnet50v2_custom_create(resnet50v2_pretrained, modelname, random_seed, learningrate, momentum, dropoutrate, L2_regularizer)        
    
    # appending parameters for record keeping
    totalparameters.append(totalparams)
    trainableparameters.append(trainableparams)
    nontrainableparameters.append(nontrainableparams)
    
    # train the model
    history, training_time_formatted = resnet50v2_custom_train(model, X_train, y_train, X_val, y_val, batchsize, epoch, directory_path_output)

    # plotting graph
    plot_graph(history, model.name, training_time_formatted, directory_path_output)

# **Saving Training Stats**

In [ ]:
# Check lengths of lists
# print("Lengths:")
# print("Model Names:", len(modelnames))
# print("Learning Rates:", len(learningrates))
# print("Momentums:", len(momentums))
# print("Dropout Rates:", len(dropoutrates))
# print("Batch Sizes:", len(batchsizes))
# print("Epochs:", len([epoch] * len(modelnames)))
# print("Total params:", len([totalparameters] * len(modelnames)))
# print("Trainable params:", len([trainableparameters] * len(modelnames)))
# print("Non-trainable params:", len([nontrainableparameters] * len(modelnames)))

data_distribution = {
    'training features': [X_train.shape] * len(modelnames),
    'training targets': [y_train.shape] * len(modelnames),
    'validation features': [X_val.shape] * len(modelnames),
    'validation targets': [y_val.shape] * len(modelnames),
    'X_train': [f'{(X_train.shape[0]/train_features.shape[0])*100:.2f}%'] * len(modelnames),
    'X_val': [f'{(X_val.shape[0]/train_features.shape[0])*100:.2f}%'] * len(modelnames),
    'y_train': [f'{(y_train.shape[0]/train_targets.shape[0])*100:.2f}%'] * len(modelnames),
    'y_val': [f'{(y_val.shape[0]/train_targets.shape[0])*100:.2f}%'] * len(modelnames)
}
model_params = {
    'Total params': totalparameters,
    'Trainable params': trainableparameters,
    'Non-trainable params': nontrainableparameters
}

# Create a DataFrame
df = pd.DataFrame({
    'Model Names': modelnames,
    'New Changes': [new_changes] * len(modelnames),
    'Random Seed': [random_seed] * len(modelnames),
    'Learning Rates': learningrates,
    'Momentums': momentums,
    'Dropout Rates': dropoutrates,
    'L2 Regularizers': L2_regularizers,
    'Batch Sizes': batchsizes,
    'Epochs': [epoch] * len(modelnames),
    'First Training Layer': [first_trainable_layer] * len(modelnames),
    **model_params,
    **data_distribution
})

# Save to Excel file
excel_filename = f'{directory_path_output}training_record_19th_batch_training.xlsx'
df.to_excel(excel_filename, index=False)

# **Evaluating All Models Performance**

In [ ]:
for i in range (0, len(learningrates)):    
    
    # loads the weights
    model.load_weights(f"/kaggle/working/resnet50v2_{i+1}_19th_batch_training.h5")
    
    result = model.evaluate(X_val, y_val)
    print(f"Evaluation of model_{i+1} on validation data")
    print(f"Test loss, Test acc: {result[0]:.4f} {result[1]*100:.2f}%\n")

# **Saving Predicted File On Test Data**

In [ ]:
# importing training data
y_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

# specifying size of the image
img_size_org = (28,28,1)

# normalizing features
y_test = y_test / 255

# converting pandas dataframe to numpy array
y_test = y_test.values

# resizing orignal images dimension and number of channels to the required dimensions and number of channels
y_test = y_test.reshape((28000,28,28))
y_test = np.expand_dims(y_test, axis=-1)
y_test = tf.image.grayscale_to_rgb(tf.convert_to_tensor(y_test))
y_test = tf.image.resize(y_test, (32,32))
y_test = y_test.numpy()
    
# loads the weights
model.load_weights(f"/kaggle/input/resnet50v2-5/resnet50v2_9_18th_batch_training.h5")
    
# making predictions on all classes | shape (28000, 10)
predictions = model.predict(y_test)
    
# assigning labels based on the predictions
predicted_labels = np.argmax(predictions, axis=1)

# Create a DataFrame with two columns: 'ImageId' and 'Label'
df = pd.DataFrame({'ImageId': range(1, len(predicted_labels) + 1), 'Label': predicted_labels})

# Export the DataFrame to a CSV file
df.to_csv('resnet50v2_9_predicted_labels_18th_batch_training.csv', index=False)

# **Error Analysis**

In [ ]:
# specifying a fixed seed for reproducibility
random_seed = 15

# file path of training data and directory path of output data
file_path_input = "/kaggle/input/digit-recognizer/train.csv"

# training ratio to split data on
training_ratio = 0.9

# splitting data into trainning and validation sets
X_train, X_val, y_train, y_val, org_indexes = transform_data_for_erroranalysis(file_path_input, training_ratio, random_seed)

# importing renset50v2 pretrained 
resnet50v2_pretrained = import_resnet50v2()
        
# freezing layers of resnet50v2
first_trainable_layer = 'conv5_block2_preact_bn'
resnet50v2_pretrained = freeze_layers(resnet50v2_pretrained, 'conv5_block2_preact_bn')
    
# create resnet50v2 with added layers
model, totalparams, trainableparams, nontrainableparams = resnet50v2_custom_create(resnet50v2_pretrained, "resnet50v2", 13, 0.001, 0.95, 0.5, 0.0001)

# loads the weights
model.load_weights("/kaggle/working/resnet50v2_2_19th_batch_training.h5")

result = model.evaluate(X_val, y_val)
print(f"Evaluation of resnet50v2-5 on validation data")
print(f"Test loss, Test acc: {result[0]:.4f} {result[1]*100:.2f}%\n")

# making predictions on all classes | shape (28000, 10)
predictions_val = model.predict(X_val)
    
# assigning labels based on the predictions
predicted_labels_val = np.argmax(predictions_val, axis=1)

print("Length of predicted labels array", len(predicted_labels_val))
print("Length of target labels array", len(y_val))

# Find the indices of True values in each row and concatenate into a flat list
true_indices = np.concatenate([np.where(row)[0] for row in y_val])

# comparing predictions and ground labels
mismatched_indexes = []
for index, (predicted, actual) in enumerate(zip(predicted_labels_val, true_indices)):
    if not np.array_equal(predicted, actual):
        mismatched_indexes.append(index)
print(f"\nPrecentange of wrong labels: {(len(mismatched_indexes)/len(predictions_val)) * 100:.2f}%")
print(f"Precentange of correct labels: {100-(len(mismatched_indexes)/len(predictions_val)) * 100:.2f}%")

In [ ]:
# Assuming X_val is your validation set and mismatched_indexes is the array of indexes
# Also, assuming each image in X_val is a 2D array (grayscale) or a 3D array (RGB)

# Define the number of columns in the plot grid
num_columns = 5

# Calculate the number of rows needed based on the number of mismatched indexes
num_rows = int(np.ceil(len(mismatched_indexes) / num_columns))

# Set up the plot grid
fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 3*num_rows))

# Flatten the axes array if the grid is not 2D
axes = axes.flatten()

# Loop through the mismatched indexes and plot the corresponding images
for i, index in enumerate(mismatched_indexes):
    image = X_val[index]  # Assuming X_val contains the images
    axes[i].imshow(image, cmap='gray')  # Change 'gray' to 'viridis' or other colormap for RGB images
    axes[i].set_title(f"Index: {index}\nPredicted: {predicted_labels_val[index]}, Actual: {true_indices[index]}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# **Saving Error Analysis Stats**

In [ ]:
# Create a DataFrame with 'Index', 'Actual', and 'Predicted' columns
data = {'Index': mismatched_indexes,
        'Actual': [true_indices[index] for index in mismatched_indexes],
        'Predicted': [predicted_labels_val[index] for index in mismatched_indexes]}

df_output = pd.DataFrame(data)

# Save the DataFrame to an Excel file
output_file_path = '/kaggle/working/error_analysis_resnet50v2_2_20th_batch_training.xlsx'  # Replace with the desired file path
df_output.to_excel(output_file_path, index=False)

# **Data Augmentation**

>**Run this code cell to get indexes of all mismatched images**

In [ ]:
print(mismatched_indexes)
print("Number of mismatched images:", len(mismatched_indexes))

>**Run this code cell to plot images - org_indexes[index] produces the orginial index of the image in training data**

In [ ]:
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

# Extract labels and pixel values
labels = train_data["label"]
pixels = train_data.drop("label", axis=1)

# Convert label values to numpy array
labels_array = np.array(labels)

# Convert pixel values to numpy array
pixels_array = np.array(pixels)

# Reshape the pixel values to 28x28 images (assuming MNIST dataset)
images = pixels_array.reshape(-1, 28, 28)

# Function to plot a specific image by index
def plot_image(index):
    plt.imshow(images[index], cmap='gray')
    plt.title(f"Label: {labels[index]}")
    plt.axis('off')
    plt.show()

# Plot a specific image by index (change the index as needed)
plot_image(int(org_indexes[3986]))

print(images.shape)

> **Run this code cell to produce the list of images that will be augmented in the next code cell**

In [ ]:
# list of indexes of images that are not needed for augmentation
mismatched_indexes_remove = [255, 314, 659, 667, 908, 955, 973, 1025, 1032, 1036, 1051, 1129, 1160, 1230, 1239, 1300, 1358, 1372, 1408, 1442, 1583, 1619, 1640, 1674, 1688, 1707, 1781, 1851, 1881, 1889, 2017, 3039, 3105, 3214, 3275, 3297, 3821, 3822, 3864, 3874, 3986, 4084]
org_indexes_mismatched_remove = []
for i, index in enumerate (mismatched_indexes_remove):
        # appending indexes of mismatched images only
        org_indexes_mismatched_remove.append(int(org_indexes[index]))

# creating a list to store only the indexes of mismatched images
org_indexes_mismatched = []
for i, index in enumerate (mismatched_indexes):
        # appending indexes of mismatched images only
        org_indexes_mismatched.append(int(org_indexes[index]))

# creating an array to store only the mismatched images and labels,
# excpet for the ones that are not needed for data augmentation
images_mismatched = []
labels_mismatched = []

for i, index in enumerate (org_indexes_mismatched):

    if index not in org_indexes_mismatched_remove:
        # x stores the image pixels and reshapes it to (1, 28, 28, 1)
        x = images[index]
        x = x.reshape((1,) + x.shape + (1,))
        y = labels_array[index]
        # appending mismatched images
        images_mismatched.append(x)
        labels_mismatched.append(y)

# converting images_mismatched list to numpy array
images_mismatched = np.array(images_mismatched)
labels_mismatched = np.array(labels_mismatched)

print(len(images_mismatched))

> **Run this code cell to produce augmented images and exported to csv file**

In [ ]:
# file path of directory where augmented images will be stored inthe form of csv file
file_path_aug = "augmented_data_resnet50v2__19th_batch_training.csv"

# using tf built-in function to produce randomly augmentded image
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False)

augmented_images = [] # this will be used to store all augmented images
total_augmented_images = 2500 # total augmented images to be created
augmentation_per_image = int(total_augmented_images/images_mismatched.shape[0]) # number of augmented images to be 
                                                                                # created per image
# this loop will be used to perform datagen.flow on all images
for i in range(0, len(images_mismatched)):
    batches = 0
    # in this loop each image will be used to produce augmented images
    # until the loop breaks
    for x_batch in datagen.flow(images_mismatched[i], batch_size=1):
        x_batch = x_batch.reshape((-1)) # reshapes x_batch to (784,)
        x_batch = np.insert(x_batch, 0, labels_mismatched[i]) # Insert label at the beginning of the array
        augmented_images.append(x_batch)
        batches += 1
        if batches > augmentation_per_image:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

augmented_images = np.array(augmented_images) # converting list to numpy array
augmented_images = augmented_images.astype(int) # converting float datatype to int

# saving augmented images to csv file
header = ",".join(["label"] + [f"pixel{i}" for i in range(augmented_images.shape[1]-1)])
np.savetxt(file_path_aug, augmented_images, delimiter=',', header=header, comments='')

> **Run this code cell to augment entire training data**

In [ ]:
# file path of directory where augmented images will be stored inthe form of csv file
train_data = "/kaggle/input/digit-recognizer/train.csv"

# file path of directory where augmented images will be stored inthe form of csv file
file_path_aug = "augmented_data_all_.csv"

# using tf built-in function to produce randomly augmentded image
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False)

augmented_images = [] # this will be used to store all augmented images

# this loop will be used to perform datagen.flow on all images
for i in range(0, images.shape[0]):
    batches = 0
    # in this loop each image will be used to produce augmented images
    # until the loop breaks
    for x_batch in datagen.flow(images[i].reshape(1,28,28,1), batch_size=1):
        x_batch = x_batch.reshape((-1)) # reshapes x_batch to (784,)
        x_batch = np.insert(x_batch, 0, labels_array[i]) # Insert label at the beginning of the array
        augmented_images.append(x_batch)
        batches += 1
        if batches > 2:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

augmented_images = np.array(augmented_images) # converting list to numpy array
augmented_images = augmented_images.astype(int) # converting float datatype to int
print(augmented_images.shape)

# saving augmented images to csv file
header = ",".join(["label"] + [f"pixel{i}" for i in range(augmented_images.shape[1]-1)])
np.savetxt(file_path_aug, augmented_images, delimiter=',', header=header, comments='')